<h1>using tutorial <a href="https://machinelearningmastery.com/how-to-interpolate-and-perform-vector-arithmetic-with-faces-using-a-generative-adversarial-network/">here</a>:</h1>

In [1]:
#basic keras GAN using celebA

from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

Using TensorFlow backend.


<p>The discriminator model takes as input one 80×80 color image an outputs a binary prediction as to whether the image is real (<em>class=1</em>) or fake (<em>class=0</em>). It is implemented as a modest convolutional neural network using best practices for GAN design such as using the <a href="https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/">LeakyReLU activation function</a> with a slope of 0.2, using a <a href="https://machinelearningmastery.com/padding-and-stride-for-convolutional-neural-networks/">2×2 stride to downsample</a>, and the <a href="https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/">adam version of stochastic gradient descent</a> with a learning rate of 0.0002 and a momentum of 0.5</p>

In [2]:
#define and compile standalone discriminator model per above
def define_discriminator(in_shape=(80,80,3)):
    #all of these 'functions' are from keras, see the first cell above
    model = Sequential()
    #normal input start
    model.add(Conv2D(128, (5,5), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    #downsample to 40x40
    model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    #downsample to 20x20
    model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    #downsample to 10x10
    model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    #downsample to 5x5
    model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    #flatten and classify
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    
    #COMPILE MODEL
    
    #optimizer
    opt = Adam(lr=0.0002, beta_1=0.5)
    #compile
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
    

The generator model takes as input a point in the latent space and outputs a single 80×80 color image.
<br>
This is achieved by using a fully connected layer to interpret the point in the latent space and provide sufficient activations that can be reshaped into many copies (in this case 128) of a low-resolution version of the output image (e.g. 5×5). This is then upsampled four times, doubling the size and quadrupling the area of the activations each time using transpose convolutional layers. The model uses best practices such as the LeakyReLU activation, a kernel size that is a factor of the stride size, and a hyperbolic tangent (tanh) activation function in the output layer.
<br>
The <em>define_generator()</em> function below defines the generator model but intentionally does not compile it as it is not trained directly, then returns the model. The size of the latent space is parameterized as a function argument.

In [3]:
#define and DON'T compile the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    #foundation for 5x5 feature maps  per above
    n_nodes = 128 * 5 * 5
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((5, 5, 128)))
    #upsample to 10x10
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    #upsample to 20x20
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    #upsample to 40x40
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    #upsample to 80x80
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    #output conv2D layer to get RGB?! -> 80x80x3
    model.add(Conv2D(3, (5,5), activation='tanh', padding='same'))
    
    return model

Next, a GAN model can be defined that combines both the generator model and the discriminator model into one larger model. This larger model will be used to train the model weights in the generator, using the output and error calculated by the discriminator model. The discriminator model is trained separately, and as such, the model weights are marked as not trainable in this larger GAN model to ensure that only the weights of the generator model are updated. This change to the trainability of the discriminator weights only has an effect when training the combined GAN model, not when training the discriminator standalone.

This larger GAN model takes as input a point in the latent space, uses the generator model to generate an image, which is fed as input to the discriminator model, then output or classified as real or fake.

The <em>define_gan()</em> function below implements this, taking the already-defined generator and discriminator models as input.

In [4]:
#define a new combined generator--discriminator model,
#the purpose of which is to update the generator kernal (weights?)
def define_gan(g_model, d_model):
    #make weights in the discriminator not trainable
    d_model.trainable = False
    #connect the two component models into a new one
    model = Sequential()
    #add generator
    model.add(g_model)
    model.add(d_model)
    #compile umbrella model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

before training, now we must import the dataset, then map its multitudinous pixel values to the range [-1,1] from their default input [0,255].
<br>
the below <em>load_real_samples()</em> function achieves this and returns the entire dataset object likewise prepared

In [5]:
#import and normalize the training dataset
def load_real_samples():
    #load the dataset
    data = load('align-80px_celebA_10000set.npz')
    X = data['arr_0']
    #convert from unsigned ints to floats
    X = X.astype('float32')
    #map (normalize) data from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X

each update to the GAN umbrella model is achieved by looking at a batch of real images and comparing them to the generated ones. thus, we need to produce a random batch of images every update.
<br>
The <em>generate_real_samples()</em> function below implements this, taking the prepared dataset as an argument, selecting and returning a random sample of face images and their corresponding class label for the discriminator, specifically class=1, indicating that they are real images.

In [6]:
#function to return random batch of real image samples
def generate_real_samples(dataset, n_samples):
    #select random instances
    #NOTE: constructor randint(low, high, size)
    ix = randint(0, dataset.shape[0], n_samples)
    #retrieve selected images
    X = dataset[ix]
    #generate 'real' class labels (literally '1')
    #NOTE: constructor ones(shape, dtype)
    #which means that the (()) here means that we are specifying an array
    #as the shape of the ones-array....slightly odd to see (x,1) but hey
    y = ones((n_samples, 1))
    #return the random real samples in the form of the images, plus
    #a single ones-array indicating their veracity
    return X, y

<p>Next, we need inputs for the generator model. These are random points from the latent space, specifically <a href="https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/">Gaussian distributed random variables</a>.</p>
<p>The <em>generate_latent_points()</em> function below implements this, taking the size of the latent space as an argument and the number of points required and returning them as a batch of input samples for the generator model.</p>

In [7]:
#make random points in (generator) latent space
#as seed inputs for generator
def generate_latent_points(latent_dim, n_samples):
    #generate points in the latent space
    #NOTE: constructor randn(d0, d1, ... dn) so the shape itself, no other params
    x_input = randn(latent_dim * n_samples)
    #reshape into a batch of inputs for the network
    #NOTE: constructor numpy.reshape(newshape[int or tuple])
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

#NOTA BENE: strangely enough, this function conjures a horribly
#undifferentiated, 1D string of numbers (latent_dim * n_samples) long,
#THEN 'reshapes' it into an array with n_samples as the first dimension..

finally, we use use the above-generated random latent points to generate new images, which then get fed into the discriminator in order to go back and train the generator.
<br><br>
The <em>generate_fake_samples()</em> function below implements this, taking the generator model and size of the latent space as arguments, then generating points in the latent space and using them as input to the generator model. The function returns the generated images and their corresponding class label for the discriminator model, specifically class=0 to indicate they are fake/generated.

In [8]:
#use the generator to generate n fake images/samples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    #generate points in latents space via above function
    x_input = generate_latent_points(latent_dim, n_samples)
    #predict outputs
    #???? i have no idea what this means but it seems to simply run
    #the model without compiling it..? LOOK INTO THIS!!
    X = g_model.predict(x_input)
    #generate 'fake' class labels (literally '0')
    #NOTE: as above, the double parens indicate an array passed as
    #the first functional argument, i.e. the shape of the zeros array
    y = zeros((n_samples, 1))
    return X, y

We are now ready to fit the GAN models.

The model is fit for 100 training epochs, which is arbitrary, as the model begins generating plausible faces after perhaps the first few epochs. A batch size of 128 samples is used, and each training epoch involves 50,000/128 or about 390 batches of real and fake samples and updates to the model.

First, the discriminator model is updated for a half batch of real samples, then a half batch of fake samples, together forming one batch of weight updates. The generator is then updated via the combined GAN model. Importantly, the class label is set to 1 or real for the fake samples. This has the effect of updating the generator toward getting better at generating real samples on the next batch.

The train() function below implements this, taking the defined models, dataset, and size of the latent dimension as arguments and parameterizing the number of epochs and batch size with default arguments.

In [9]:
#function to TRAIN THE GAN MODEL! per above
def train(g_model, d_model, gan_model, dataset, latent_dim, render_latent, n_epochs=10, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    
    #manually loop epochs
    for i in range(n_epochs):
        #loop batches over the training set
        for j in range(bat_per_epo):
            #get half batch of random real samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            #update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            #generate half batch of fake samples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            #update discriminator model weights again
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            #prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            #create inverted labels for the fake samples (i dont get this)
            y_gan = ones((n_batch, 1))
            #update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            
            #summarize loss on this batch
            print('>>>>%d, %d/%d, d1=%.3f, d2=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
            
            #render single latent
            render_plot(g_model, render_latent, i, j)
            
        #evaluate model performance, sometimes
        if (i+1) % 1 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

next section are two handy functions: one to save a plot of n images from the generator, from the tutorial; and one to render a single (constant) latent point, which is defined below

In [10]:
#create and save a plot of generated images
def save_plot(examples, epoch, n=10):
    #scale from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    #plot images
    for i in range(n * n):
        #define subplot
        pyplot.subplot(n, n, 1 + i)
        #turn off axes
        pyplot.axis('off')
        #plot raw pixel data
        pyplot.imshow(examples[i])
    
    #save plot to file
    filename = 'generated_plot_e%03d.png' % epoch
    pyplot.savefig(filename)
    pyplot.close()

In [11]:
#create and save a plot of a single latent point, saved with #e## filename
def render_plot(g_model, render_latent, epoch, batch):
    #as in generate_fake_samples()
    render = g_model.predict(render_latent)
    #scale from [-1,1] to [0,1]
    render = (render + 1) / 2
    #plot render
    pyplot.axis('off')
    pyplot.imshow(render[0])
    
    #save plot to file
    filename = 'render01\\render_%03de%03d.png' % (epoch, batch)
    pyplot.savefig(filename)
    pyplot.close()

The <em>summarize_performance()</em> function generates samples and evaluates the performance of the discriminator on real and fake samples. The classification accuracy is reported and might provide insight into model performance. The <em>save_plot()</em> is called to create and save a plot of the generated images, and then the model is saved to a file.

In [12]:
#function to evaluate the discriminator, plot generated images, and save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    #prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    #evaluate discriminator on real samples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    #prepare fake samples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    #evaluate discriminator on fake samples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    #summarize and print
    print('discriminator accuracy: real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    #save plot
    save_plot(x_fake, epoch)
    #save generator model tile file
    filename = 'generator_model_%03d.h5' % epoch
    g_model.save(filename)

finally, this last cell can be used to define the size of the latent space, define all three models, and train them

In [13]:
#size of latent space
latent_dim = 100
#create discriminator
d_model = define_discriminator()

In [14]:
#create generator
g_model = define_generator(latent_dim)

In [15]:
#create gan umbrella model
gan_model = define_gan(g_model, d_model)

In [16]:
#load dataset
dataset = load_real_samples()

now i will pause and create a system for <em>rendering</em> the training process as i have done in my other GAN experiments

In [17]:
render_latent = generate_latent_points(latent_dim, 1)

In [18]:
#TRAIN!!!!
train(g_model, d_model, gan_model, dataset, latent_dim, render_latent)

C:\Users\jjeff\Anaconda3\envs\neuGAN\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\jjeff\Anaconda3\envs\neuGAN\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>>>>1, 1/78, d1=0.691, d2=0.696, g=0.692


C:\Users\jjeff\Anaconda3\envs\neuGAN\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>>>>1, 2/78, d1=0.518, d2=0.698, g=0.691
>>>>1, 3/78, d1=0.172, d2=0.717, g=0.680
>>>>1, 4/78, d1=0.011, d2=0.757, g=0.675
>>>>1, 5/78, d1=0.003, d2=0.767, g=0.684
>>>>1, 6/78, d1=0.009, d2=0.750, g=0.699
>>>>1, 7/78, d1=0.028, d2=0.728, g=0.716
>>>>1, 8/78, d1=0.037, d2=0.710, g=0.736
>>>>1, 9/78, d1=0.026, d2=0.663, g=0.774
>>>>1, 10/78, d1=0.008, d2=0.628, g=0.863
>>>>1, 11/78, d1=0.006, d2=0.573, g=1.000
>>>>1, 12/78, d1=0.017, d2=0.518, g=1.095
>>>>1, 13/78, d1=0.000, d2=0.399, g=1.535
>>>>1, 14/78, d1=0.000, d2=0.225, g=2.442
>>>>1, 15/78, d1=0.000, d2=0.072, g=3.787
>>>>1, 16/78, d1=0.250, d2=0.044, g=3.456
>>>>1, 17/78, d1=0.000, d2=0.031, g=4.037
>>>>1, 18/78, d1=0.000, d2=0.027, g=4.576
>>>>1, 19/78, d1=0.000, d2=0.309, g=8.168
>>>>1, 20/78, d1=0.000, d2=0.000, g=12.128
>>>>1, 21/78, d1=0.000, d2=0.000, g=9.567
>>>>1, 22/78, d1=0.000, d2=0.635, g=17.497
>>>>1, 23/78, d1=0.061, d2=0.000, g=13.646
>>>>1, 24/78, d1=0.316, d2=0.014, g=4.987
>>>>1, 25/78, d1=0.000, d2=22.229, g=2.

>>>>3, 41/78, d1=0.053, d2=0.077, g=4.974
>>>>3, 42/78, d1=0.096, d2=0.148, g=5.325
>>>>3, 43/78, d1=0.099, d2=0.132, g=5.600
>>>>3, 44/78, d1=0.372, d2=0.379, g=5.737
>>>>3, 45/78, d1=0.435, d2=0.454, g=5.679
>>>>3, 46/78, d1=0.239, d2=0.197, g=5.813
>>>>3, 47/78, d1=0.362, d2=0.366, g=5.467
>>>>3, 48/78, d1=0.381, d2=0.176, g=4.813
>>>>3, 49/78, d1=0.087, d2=0.167, g=5.420
>>>>3, 50/78, d1=0.146, d2=0.155, g=4.969
>>>>3, 51/78, d1=0.169, d2=0.114, g=4.136
>>>>3, 52/78, d1=0.138, d2=0.128, g=3.542
>>>>3, 53/78, d1=0.068, d2=0.117, g=3.751
>>>>3, 54/78, d1=0.059, d2=0.055, g=3.874
>>>>3, 55/78, d1=0.019, d2=0.083, g=4.003
>>>>3, 56/78, d1=0.002, d2=0.033, g=4.278
>>>>3, 57/78, d1=0.018, d2=0.054, g=4.057
>>>>3, 58/78, d1=0.007, d2=0.061, g=3.685
>>>>3, 59/78, d1=0.055, d2=0.194, g=3.650
>>>>3, 60/78, d1=0.087, d2=0.156, g=3.657
>>>>3, 61/78, d1=0.001, d2=0.099, g=3.995
>>>>3, 62/78, d1=0.015, d2=0.169, g=3.935
>>>>3, 63/78, d1=0.058, d2=0.215, g=4.547
>>>>3, 64/78, d1=0.173, d2=0.050, 

>>>>6, 3/78, d1=0.286, d2=0.161, g=2.997
>>>>6, 4/78, d1=0.280, d2=0.205, g=2.985
>>>>6, 5/78, d1=0.169, d2=0.158, g=3.214
>>>>6, 6/78, d1=0.285, d2=0.130, g=2.923
>>>>6, 7/78, d1=0.127, d2=0.241, g=3.348
>>>>6, 8/78, d1=0.165, d2=0.080, g=3.569
>>>>6, 9/78, d1=0.235, d2=0.372, g=4.202
>>>>6, 10/78, d1=0.442, d2=0.091, g=3.120
>>>>6, 11/78, d1=0.264, d2=0.591, g=4.035
>>>>6, 12/78, d1=0.634, d2=0.252, g=2.857
>>>>6, 13/78, d1=0.392, d2=0.288, g=2.598
>>>>6, 14/78, d1=0.282, d2=0.190, g=2.822
>>>>6, 15/78, d1=0.244, d2=0.268, g=3.187
>>>>6, 16/78, d1=0.285, d2=0.216, g=3.046
>>>>6, 17/78, d1=0.196, d2=0.176, g=3.084
>>>>6, 18/78, d1=0.279, d2=0.324, g=3.395
>>>>6, 19/78, d1=0.317, d2=0.141, g=3.049
>>>>6, 20/78, d1=0.272, d2=0.221, g=2.612
>>>>6, 21/78, d1=0.223, d2=0.227, g=2.872
>>>>6, 22/78, d1=0.263, d2=0.220, g=3.061
>>>>6, 23/78, d1=0.220, d2=0.142, g=2.953
>>>>6, 24/78, d1=0.209, d2=0.161, g=2.658
>>>>6, 25/78, d1=0.120, d2=0.182, g=2.973
>>>>6, 26/78, d1=0.197, d2=0.153, g=2.819

>>>>8, 43/78, d1=0.239, d2=0.189, g=3.856
>>>>8, 44/78, d1=0.183, d2=0.199, g=3.917
>>>>8, 45/78, d1=0.085, d2=0.188, g=3.922
>>>>8, 46/78, d1=0.391, d2=0.242, g=3.334
>>>>8, 47/78, d1=0.373, d2=0.219, g=3.461
>>>>8, 48/78, d1=0.232, d2=0.155, g=3.770
>>>>8, 49/78, d1=0.148, d2=0.212, g=3.747
>>>>8, 50/78, d1=0.310, d2=0.156, g=3.199
>>>>8, 51/78, d1=0.217, d2=0.216, g=3.678
>>>>8, 52/78, d1=0.253, d2=0.146, g=4.172
>>>>8, 53/78, d1=0.222, d2=0.107, g=3.540
>>>>8, 54/78, d1=0.139, d2=0.157, g=3.309
>>>>8, 55/78, d1=0.189, d2=0.122, g=3.152
>>>>8, 56/78, d1=0.100, d2=0.209, g=3.863
>>>>8, 57/78, d1=0.110, d2=0.077, g=3.986
>>>>8, 58/78, d1=0.189, d2=0.157, g=3.528
>>>>8, 59/78, d1=0.169, d2=0.142, g=3.549
>>>>8, 60/78, d1=0.294, d2=0.143, g=3.240
>>>>8, 61/78, d1=0.151, d2=0.175, g=3.694
>>>>8, 62/78, d1=0.236, d2=0.109, g=3.846
>>>>8, 63/78, d1=0.255, d2=0.111, g=3.643
>>>>8, 64/78, d1=0.106, d2=0.158, g=4.426
>>>>8, 65/78, d1=0.281, d2=0.093, g=3.389
>>>>8, 66/78, d1=0.126, d2=0.105, 